In [ ]:
# Code contribution: Abhay & Reng
import urllib.request
from urllib.parse import urlparse
from typing import Text
from bs4.element import SoupStrainer
import requests
from bs4 import BeautifulSoup
from dateutil.parser import parse
import concurrent.futures
import pandas as pd
import os
from tqdm import tqdm

class IMDBScraper:    
    def __init__(self, url):
        self.url = url
        self.download_page()

    def download_page(self):
       # method for downloading the hotel page
        self.page = requests.get(self.url).text

    def scrape_data(self):
        #method for scraping out movie title and description
        request = urllib.request.Request(self.page)
        try:
            response = urllib.request.urlopen(self.url)

            soup = BeautifulSoup(self.page, "html.parser") 
            title = soup.find(attrs={'data-testid':"hero-title-block__title"}).string if soup.find(attrs={'data-testid':"hero-title-block__title"}) != None else ""
            mdata = soup.find_all(class_="sc-8c396aa2-2 itZqyK")
            if mdata != None: 
                year = mdata[0].string 
            description = soup.find("span", {"data-testid": "plot-xl"}).text
            dur = soup.find("ul", {"data-testid": "hero-title-block__metadata"}) 
            if dur != None:
                duration = dur.find_all('li')[-1].text
            else: 
                duration = ""

          # better: we can find top credits element -> find first child -> find siblings 

            top = soup.find(class_="sc-127a9a54-1 doAoXk sc-fa02f843-2 dwQKsL")
            director = [] 
            writer = []
            stars = []

            if top != None: 
                top_credits = soup.find(class_="sc-127a9a54-1 doAoXk sc-fa02f843-2 dwQKsL").find('ul')        

            if (len(top_credits.contents) > 0):
                for x in top_credits.contents: 
                    if x.select('a[href*="ov_dr"]'): 
                        list_dir = x.select('a[href$="ov_dr"]')
                        for x in list_dir: 
                            director.append(x.text)
                    elif x.select('a[href*="ov_wr"]'): 
                        list_wr = x.select('a[href$="ov_wr"]')
                        for x in list_wr: 
                            writer.append(x.text)
                    elif x.select('a[href*="ov_st"]'): 
                        list_st = x.select('a[href$="ov_st"]')
                        for x in list_st: 
                            stars.append(x.text)

            ratings = soup.find(class_="sc-7ab21ed2-1 jGRxWM").string if soup.find(class_="sc-7ab21ed2-1 jGRxWM") != None else ""
            return [title, year, description, duration, director, writer, stars, ratings]
        except urllib.error.HTTPError as e:
            print("404 Error")
            print(self.url)
            return ["", "", "", "", "", "", "", ""]
        except: 
            print("Other format errors")
            print(self.url)
            return ["", "", "", "", "", "", "", ""]
# beware what if no value? 

def f(tmp): 
    return IMDBScraper(tmp).scrape_data()

In [ ]:
from google.colab import drive 
drive.mount('/content/drive',force_remount=True) 

Mounted at /content/drive


In [ ]:
import os 
import pandas as pd 

# https://stackoverflow.com/questions/54351852/accessing-shared-with-me-with-colab -> everyone has to do this once. 
os.chdir("/content/drive/MyDrive/Web Mining")

In [ ]:
ratings = pd.read_csv("./ml-1m/ratings.dat", sep="::", names=["UserID", "MovieID", "Rating", "Timestamp"], skipinitialspace=True)
ratings

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [ ]:
movies = pd.read_csv("./movie_final_full_dataset.csv")

In [ ]:
ratings_missing = (ratings.merge(movies, on = ('MovieID'), how='left', indicator = True)
                 .query('_merge == "left_only"')
                .drop('_merge',1))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
ratings_missing

,UserID,MovieID,Rating,Timestamp,Title,Genres,ImdbID,Links,title,year,description,duration,director,writer,stars,ratings
937,10,1294,4,978225952,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2763,22,1205,4,978136836,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3348,25,1205,5,978132441,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3793,27,1294,5,978129509,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3953,29,3366,2,978136307,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999667,6037,1294,5,956719383,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
999836,6039,1294,3,956705300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
999910,6040,978,4,957717557,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000086,6040,1294,4,957716949,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
ratings_new = ratings[~ratings.MovieID.isin(ratings_missing.MovieID)]
ratings_new.reset_index(inplace=True)

In [ ]:
ratings_new.drop('index',axis=1,inplace=True)
ratings_new

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
997251,6040,1091,1,956716541
997252,6040,1094,5,956704887
997253,6040,562,5,956704746
997254,6040,1096,4,956715648


In [ ]:
ratings_new.to_csv("./ratings_new.csv", index=False)

In [ ]:
# Merged Movies dataset
movies_names = pd.read_csv('Reng/merged_movies.csv', header= 0, skipinitialspace=True, index_col = 0) 
movies_names

,MovieID,Title,Genres,ImdbID,Links
0,1,Toy Story (1995),Animation|Children's|Comedy,114709,https://www.imdb.com/title/tt0114709/
1,2,Jumanji (1995),Adventure|Children's|Fantasy,113497,https://www.imdb.com/title/tt0113497/
2,3,Grumpier Old Men (1995),Comedy|Romance,113228,https://www.imdb.com/title/tt0113228/
3,4,Waiting to Exhale (1995),Comedy|Drama,114885,https://www.imdb.com/title/tt0114885/
4,5,Father of the Bride Part II (1995),Comedy,113041,https://www.imdb.com/title/tt0113041/
...,...,...,...,...,...
3854,3948,Meet the Parents (2000),Comedy,212338,https://www.imdb.com/title/tt0212338/
3855,3949,Requiem for a Dream (2000),Drama,180093,https://www.imdb.com/title/tt0180093/
3856,3950,Tigerland (2000),Drama,170691,https://www.imdb.com/title/tt0170691/
3857,3951,Two Family House (2000),Drama,202641,https://www.imdb.com/title/tt0202641/


In [ ]:
# Movies from 20m
movies_20m = pd.read_csv("./ml-20m/movies.csv") 
movies_20m = movies_20m.rename(columns={"movieId": "MovieID", "title": "Title", "genres": "Genres"})

In [ ]:
movies_names['Genres'] = movies_20m['Genres'][0:3859]

In [ ]:
movies_names

,MovieID,Title,Genres,ImdbID,Links
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,https://www.imdb.com/title/tt0114709/
1,2,Jumanji (1995),Adventure|Children|Fantasy,113497,https://www.imdb.com/title/tt0113497/
2,3,Grumpier Old Men (1995),Comedy|Romance,113228,https://www.imdb.com/title/tt0113228/
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,114885,https://www.imdb.com/title/tt0114885/
4,5,Father of the Bride Part II (1995),Comedy,113041,https://www.imdb.com/title/tt0113041/
...,...,...,...,...,...
3854,3948,Meet the Parents (2000),Comedy,212338,https://www.imdb.com/title/tt0212338/
3855,3949,Requiem for a Dream (2000),Drama,180093,https://www.imdb.com/title/tt0180093/
3856,3950,Tigerland (2000),Drama,170691,https://www.imdb.com/title/tt0170691/
3857,3951,Two Family House (2000),Drama,202641,https://www.imdb.com/title/tt0202641/


In [ ]:
df = pd.DataFrame(columns=['MovieID',	'Title',	'Genres',	'ImdbID',	'Links',	'title',	'year',	'description',	'duration',	'director',	'writer',	'stars', 'ratings'])
len(df)

0

In [ ]:
# Scraping data
from multiprocessing.pool import ThreadPool

movie2 = movies_names[0:500]
with ThreadPool(32) as p:
  # result = p.map(f, merged_movies.itertuples())
  movie2[['title', 'year', 'description', 'duration', 'director', 'writer', 'stars', 'ratings']] = pd.DataFrame(p.map(f, movie2['Links'].tolist()), index = movie2.index)
  #result = p.map(f, movie2.loc[0:500].itertuples())
df = df.append(movie2).reset_index(drop=True)
del(movie2)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [ ]:

movie2 = movies_names[500:1000]
with ThreadPool(32) as p:
  # result = p.map(f, merged_movies.itertuples())
  movie2[['title', 'year', 'description', 'duration', 'director', 'writer', 'stars', 'ratings']] = pd.DataFrame(p.map(f, movie2['Links'].tolist()), index = movie2.index)
df = df.append(movie2).reset_index(drop=True)
del(movie2)

404 Error
https://www.imdb.com/title/tt0118114/
Other format errors
https://www.imdb.com/title/tt0116485/
Other format errors
https://www.imdb.com/title/tt0125877/
Other format errors
https://www.imdb.com/title/tt0038426/
Other format errors
https://www.imdb.com/title/tt0105737/
Other format errors
https://www.imdb.com/title/tt0113610/
Other format errors
https://www.imdb.com/title/tt0113147/
Other format errors
https://www.imdb.com/title/tt0116559/


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [ ]:

movie2 = movies_names[1000:1500]
with ThreadPool(32) as p:
  # result = p.map(f, merged_movies.itertuples())
  movie2[['title', 'year', 'description', 'duration', 'director', 'writer', 'stars', 'ratings']] = pd.DataFrame(p.map(f, movie2['Links'].tolist()), index = movie2.index)
  #result = p.map(f, movie2.loc[0:500].itertuples())
df = df.append(movie2).reset_index(drop=True)
del(movie2)

Other format errors
https://www.imdb.com/title/tt0092281/
Other format errors
https://www.imdb.com/title/tt0115548/
Other format errors
https://www.imdb.com/title/tt0107314/


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [ ]:

movie2 = movies_names[1500:2000]
with ThreadPool(32) as p:
  # result = p.map(f, merged_movies.itertuples())
  movie2[['title', 'year', 'description', 'duration', 'director', 'writer', 'stars', 'ratings']] = pd.DataFrame(p.map(f, movie2['Links'].tolist()), index = movie2.index)
  #result = p.map(f, movie2.loc[0:500].itertuples())
df = df.append(movie2).reset_index(drop=True)
del(movie2)


Other format errors
https://www.imdb.com/title/tt0133090/
Other format errors
https://www.imdb.com/title/tt0116490/


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [ ]:

movie2 = movies_names[2000:2500]
with ThreadPool(32) as p:
  # result = p.map(f, merged_movies.itertuples())
  movie2[['title', 'year', 'description', 'duration', 'director', 'writer', 'stars', 'ratings']] = pd.DataFrame(p.map(f, movie2['Links'].tolist()), index = movie2.index)
  #result = p.map(f, movie2.loc[0:500].itertuples())
df = df.append(movie2).reset_index(drop=True)
del(movie2)


404 Error
https://www.imdb.com/title/tt0087690/
404 Error
https://www.imdb.com/title/tt0134948/


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [ ]:

df.to_csv("./Abhay/tmp2.csv", index=False)

In [ ]:
df = pd.read_csv("./Abhay/tmp2.csv")

,MovieID,Title,Genres,ImdbID,Links,title,year,description,duration,director,writer,stars,ratings
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,https://www.imdb.com/title/tt0114709/,Toy Story,1995,A cowboy doll is profoundly threatened and jea...,1h 21m,['John Lasseter'],"['John Lasseter', 'Pete Docter', 'Andrew Stant...","['Tom Hanks', 'Tim Allen', 'Don Rickles']",8.3
1,2,Jumanji (1995),Adventure|Children|Fantasy,113497,https://www.imdb.com/title/tt0113497/,Jumanji,1995,When two kids find and play a magical board ga...,1h 44m,['Joe Johnston'],"['Jonathan Hensleigh', 'Greg Taylor', 'Jim Str...","['Robin Williams', 'Kirsten Dunst', 'Bonnie Hu...",7.0
2,3,Grumpier Old Men (1995),Comedy|Romance,113228,https://www.imdb.com/title/tt0113228/,Grumpier Old Men,1995,John and Max resolve to save their beloved bai...,1h 41m,['Howard Deutch'],['Mark Steven Johnson'],"['Walter Matthau', 'Jack Lemmon', 'Ann-Margret']",6.6
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,114885,https://www.imdb.com/title/tt0114885/,Waiting to Exhale,1995,"Based on Terry McMillan's novel, this film fol...",2h 4m,['Forest Whitaker'],"['Terry McMillan', 'Ronald Bass']","['Whitney Houston', 'Angela Bassett', 'Loretta...",5.9
4,5,Father of the Bride Part II (1995),Comedy,113041,https://www.imdb.com/title/tt0113041/,Father of the Bride Part II,1995,George Banks must deal not only with the pregn...,1h 46m,['Charles Shyer'],"['Albert Hackett', 'Frances Goodrich', 'Nancy ...","['Steve Martin', 'Diane Keaton', 'Martin Short']",6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,2580,Go (1999),Comedy|Crime,139239,https://www.imdb.com/title/tt0139239/,Go,1999,The aftermath of a drug deal as told from thre...,1h 42m,['Doug Liman'],['John August'],"['Sarah Polley', 'Jay Mohr', 'Scott Wolf']",7.2
2496,2581,Never Been Kissed (1999),Comedy|Romance,151738,https://www.imdb.com/title/tt0151738/,Never Been Kissed,1999,A newspaper reporter enrolls in high school as...,1h 47m,['Raja Gosnell'],"['Abby Kohn', 'Marc Silverstein']","['Drew Barrymore', 'David Arquette', 'Michael ...",6.0
2497,2582,Twin Dragons (Shuang long hui) (1992),Action|Comedy,105399,https://www.imdb.com/title/tt0105399/,Twin Dragons,1992,"Identical twins are separated at birth, one be...",1h 44m,"['Ringo Lam', 'Hark Tsui']","['Barry Wong', 'Hark Tsui', ""Tung Cho 'Joe' Ch...","['Jackie Chan', 'Maggie Cheung', 'Teddy Robin ...",6.4
2498,2583,Cookie's Fortune (1999),Comedy|Drama,126250,https://www.imdb.com/title/tt0126250/,Cookie's Fortune,1999,Conflict arises in the small town of Holly Spr...,1h 58m,['Robert Altman'],['Anne Rapp'],"['Glenn Close', 'Julianne Moore', 'Liv Tyler']",6.8


In [ ]:
from multiprocessing.pool import ThreadPool

movie2 = movies_names[2500:3000]
with ThreadPool(32) as p:
  # result = p.map(f, merged_movies.itertuples())
  movie2[['title', 'year', 'description', 'duration', 'director', 'writer', 'stars', 'ratings']] = pd.DataFrame(p.map(f, movie2['Links'].tolist()), index = movie2.index)
  #result = p.map(f, movie2.loc[0:500].itertuples())
df = df.append(movie2).reset_index(drop=True)
del(movie2)


Other format errors
https://www.imdb.com/title/tt0341315/
Other format errors
https://www.imdb.com/title/tt0168950/
404 Error
https://www.imdb.com/title/tt0081454/
Other format errors
https://www.imdb.com/title/tt0150574/
404 Error
https://www.imdb.com/title/tt0154827/


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [ ]:

movie2 = movies_names[3000:3500]
with ThreadPool(32) as p:
  # result = p.map(f, merged_movies.itertuples())
  movie2[['title', 'year', 'description', 'duration', 'director', 'writer', 'stars', 'ratings']] = pd.DataFrame(p.map(f, movie2['Links'].tolist()), index = movie2.index)
  #result = p.map(f, movie2.loc[0:500].itertuples())
df = df.append(movie2).reset_index(drop=True)
del(movie2)


Other format errors
https://www.imdb.com/title/tt0154352/


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [ ]:

movie2 = movies_names[3500:3859]
with ThreadPool(32) as p:
  # result = p.map(f, merged_movies.itertuples())
  movie2[['title', 'year', 'description', 'duration', 'director', 'writer', 'stars', 'ratings']] = pd.DataFrame(p.map(f, movie2['Links'].tolist()), index = movie2.index)
  #result = p.map(f, movie2.loc[0:500].itertuples())
df = df.append(movie2).reset_index(drop=True)
del(movie2)

# with ThreadPool(32) as p:
#   # result = p.map(f, merged_movies.itertuples())
#   movie2[['title', 'year', 'description', 'duration', 'dir', 'wr', 'st', 'ratings']] = pd.DataFrame(p.map(f, movie2['Links'].loc[101:200].tolist()), index = movie2[101:201].index)


# with ThreadPool(50) as p:
#   movie2[['title', 'year', 'duration', 'dir', 'wr', 'st', 'ratings']] = pd.DataFrame(p.map(f, movie2['Links'].loc[1001:1500].tolist()), index = movie2.index)



/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df.isna().sum()

MovieID         0
Title           0
Genres          0
ImdbID          0
Links           0
title          15
year           15
description    15
duration       15
director       15
writer         15
stars          15
ratings        15
dtype: int64

In [ ]:

df.to_csv("./Abhay/movie_complete.csv", index=False)

In [ ]:
df = pd.read_csv("./Abhay/movie_full_dataset.csv", encoding="ISO-8859-1", skipinitialspace=True)

In [ ]:
df['year'].value_counts()

1995    344
1996    341
1998    330
1997    323
1999    281
       ... 
1923      2
1914      1
1920      1
1918      1
1921      1
Name: year, Length: 83, dtype: int64

In [ ]:
df.isna().sum()

MovieID        0
Title          0
Genres         0
ImdbID         0
Links          0
title          0
year           0
description    3
duration       2
director       0
writer         0
stars          0
ratings        4
dtype: int64